# Segmentation 3D

## 1. Prerequisites

### 1.1 Load cloud point of voxel centers

In [5]:
import cv2

from openalea.phenomenal.data import (
    plant_1_voxel_grid,
    plant_2_voxel_grid,
    plant_1_images_binarize,
    plant_1_calibration_camera_side,
    plant_1_calibration_camera_top)

from openalea.phenomenal.display.notebook import *

from openalea.phenomenal.object import (
    VoxelSkeleton,
    VoxelGrid,
    VoxelSegmentation,
    ImageView)

from openalea.phenomenal.multi_view_reconstruction import *

from openalea.phenomenal.segmentation_3D import (
    voxel_graph_from_voxel_grid,
    skeletonize,
    segment_reduction,
    maize_segmentation,
    maize_analysis)

In [6]:
def routine_select_ref_angle(image_views):

    max_len = 0
    image_view_max = None
    for i, iv in enumerate(image_views):

        x_pos, y_pos, x_len, y_len = cv2.boundingRect(cv2.findNonZero(iv.image))

        if x_len > max_len:
            max_len = x_len
            image_view_max = iv

    image_view_max.image_ref = image_view_max.image

    return image_views

In [7]:
images = plant_1_images_binarize()

calibration_side = plant_1_calibration_camera_side()
calibration_top = plant_1_calibration_camera_top()

# Select images
image_views = list()
for angle in range(0, 360, 30):
    projection = calibration_side.get_projection(angle)
    image_views.append(ImageView(images[angle], projection, inclusive=False))

image_views = routine_select_ref_angle(image_views)

projection = calibration_top.get_projection(0)
image_views.append(ImageView(images[-1], projection, inclusive=True))

voxels_size = 8
error_tolerance = 1
vg = reconstruction_3d(image_views,
                       voxels_size=voxels_size,
                       error_tolerance=error_tolerance)

## Display voxel grid

In [8]:
show_voxel_grid(vg)

A Jupyter Widget

## Skeletonization

Compute the skeleton of the plant voxelgrid

In [9]:
import time
t0 = time.time()

vg.voxels_position = map(tuple, list(vg.voxels_position))
vg.voxels_size = int(vg.voxels_size)

subgraph = None
voxel_graph = voxel_graph_from_voxel_grid(vg, connect_all_point=True)
voxel_skeleton = skeletonize(voxel_graph.graph,
                             voxel_graph.voxels_size,
                             subgraph=subgraph)

### Display it

In [10]:
show_skeleton(voxel_skeleton)

A Jupyter Widget

# Voxel segment reduction

In [11]:
# Select images
image_views = list()
for angle in range(0, 360, 30):
    projection = calibration_side.get_projection(angle)
    image_views.append(ImageView(images[angle],
                                 projection,
                                 inclusive=False))

voxel_skeleton_reduced = segment_reduction(voxel_skeleton,
                                   image_views,
                                   tolerance=4)

### Display it

In [12]:
show_skeleton(voxel_skeleton_reduced, color="green")

A Jupyter Widget

# Segmentation Maize

In [13]:
vms = maize_segmentation(voxel_skeleton, voxel_graph)
filename = 'voxel_maize_segmentation.json'
vms.write_to_json_gz(filename)
vms = VoxelSegmentation.read_from_json_gz(filename)

set([(156.0, -100.0, -420.0)])
set([(156.0, -100.0, -420.0)])
set([(156.0, -100.0, -420.0)])
set([(156.0, -100.0, -420.0)])
set([(156.0, -100.0, -420.0)])
set([(-220.0, 196.0, -380.0)])
set([(-220.0, 196.0, -380.0)])
set([(-220.0, 196.0, -380.0)])
set([(-220.0, 196.0, -380.0)])
set([(-220.0, 196.0, -380.0)])
set([(356.0, 276.0, -316.0)])
set([(356.0, 276.0, -316.0)])
set([(356.0, 276.0, -316.0)])
set([(356.0, 276.0, -316.0)])
set([(356.0, 276.0, -316.0)])
set([(-348.0, -420.0, -228.0)])
set([(-348.0, -420.0, -228.0)])
set([(-348.0, -420.0, -228.0)])
set([(-348.0, -420.0, -228.0)])
set([(-348.0, -420.0, -228.0)])
set([(-380.0, -460.0, 116.0)])
set([(-380.0, -460.0, 116.0)])
set([(-380.0, -460.0, 116.0)])
set([(-380.0, -460.0, 116.0)])
set([(-380.0, -460.0, 116.0)])
set([(444.0, 316.0, -76.0)])
set([(444.0, 316.0, -76.0)])
set([(444.0, 316.0, -76.0)])
set([(444.0, 316.0, -76.0)])
set([(444.0, 316.0, -76.0)])


In [14]:
show_segmentation(vms)

A Jupyter Widget

In [15]:
vmsi = maize_analysis(vms)
file_prefix = 'voxel_maize_segmentation_info'
vmsi.write_to_json_gz(file_prefix)
vmsi = VoxelSegmentation.read_from_json_gz(file_prefix)

In [16]:
show_segmentation(vms, size_ratio=0.5)

A Jupyter Widget